In [47]:
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
import string
import pandas as pd
import nltk 
import numpy as np
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
df_business = pd.read_csv('../data_snapshot/business_snap.csv')
df_review = pd.read_csv('../data_snapshot/review_snap.csv')
print(df_business)

TypeError: Passing a bool to header is invalid. Use header=None for no header or header=int or list-like of ints to specify the row(s) making up the column names

In [12]:
df_business.dropna(inplace=True)
df_business.isna().sum()

Unnamed: 0    0
0             0
1             0
2             0
3             0
4             0
5             0
6             0
7             0
8             0
9             0
10            0
11            0
dtype: int64

In [13]:
df_business.columns

<class 'pandas.core.frame.DataFrame'>
Index: 339 entries, 0 to 471
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  339 non-null    int64  
 1   0           339 non-null    object 
 2   1           339 non-null    object 
 3   2           339 non-null    object 
 4   3           339 non-null    object 
 5   4           339 non-null    int64  
 6   5           339 non-null    float64
 7   6           339 non-null    float64
 8   7           339 non-null    float64
 9   8           339 non-null    float64
 10  9           339 non-null    float64
 11  10          339 non-null    object 
 12  11          339 non-null    object 
dtypes: float64(5), int64(2), object(6)
memory usage: 37.1+ KB


In [27]:
review_user = df_review[['0','1','2','4']]
review_user

,0,1,2,4
0,1XyJMWQ0fgWszG69vh3INA,Was seated and waited 30 mins without even bei...,1,WRUIUIVkz7F5IiH7XI8QcA
1,H4FhOjFivi4aMxeddxMPGg,ROASTED CHICKEN & GOAT CHEESE PANINI \n16.50 ~...,5,y0wA70QDTUeKS_VyLw7A7A
2,qLzO2mwYN3ti8dvAfpxePw,All the vibes!! We dropped in on a Saturday w...,4,LDoa3UXbySRr0b2pcwrlnw
3,dONpgM2St8gaCZdnaWiTKw,I was so excited about the Food Truck Fest whe...,5,7DCwwpe524F2AqZqTUGExg
4,V4d7QNZtIbCfMXYcgR5LhA,This is the best Mexican restaurant in Gahanna...,5,7v5WWyFCONwphGxWqlncEw
...,...,...,...,...
1345,ignrbLkzTKrGaaL4jQ1lvA,The food is really incredible. I came here for...,5,GT_0_cScBHlj35KtTd7hSg
1346,yYBiqk7BeLP97LgnkTq4vA,You know how to tell the best ethnic restauran...,5,q1XC5SIh7EGTb4B_FYlsxg
1347,98Lkx4R4IuCUhntxFZyyAw,Good food but sometimes takes awhile to be ser...,3,qUMpWX-1mZsR18QRjABUsw
1348,iGx_o5RTELbevlqq2rBiXw,Great food truck. Met them at the Food Truck T...,5,DW1ddLY6wzPrbWQXPjWbPQ


In [25]:
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(' '.join(s))
print(stop)

['i', 'm e', 'm y', 'm y s e l f', 'w e', 'o u r', 'o u r s', 'o u r s e l v e s', 'y o u', 'y o u r e', 'y o u v e', 'y o u l l', 'y o u d', 'y o u r', 'y o u r s', 'y o u r s e l f', 'y o u r s e l v e s', 'h e', 'h i m', 'h i s', 'h i m s e l f', 's h e', 's h e s', 'h e r', 'h e r s', 'h e r s e l f', 'i t', 'i t s', 'i t s', 'i t s e l f', 't h e y', 't h e m', 't h e i r', 't h e i r s', 't h e m s e l v e s', 'w h a t', 'w h i c h', 'w h o', 'w h o m', 't h i s', 't h a t', 't h a t l l', 't h e s e', 't h o s e', 'a m', 'i s', 'a r e', 'w a s', 'w e r e', 'b e', 'b e e n', 'b e i n g', 'h a v e', 'h a s', 'h a d', 'h a v i n g', 'd o', 'd o e s', 'd i d', 'd o i n g', 'a', 'a n', 't h e', 'a n d', 'b u t', 'i f', 'o r', 'b e c a u s e', 'a s', 'u n t i l', 'w h i l e', 'o f', 'a t', 'b y', 'f o r', 'w i t h', 'a b o u t', 'a g a i n s t', 'b e t w e e n', 'i n t o', 't h r o u g h', 'd u r i n g', 'b e f o r e', 'a f t e r', 'a b o v e', 'b e l o w', 't o', 'f r o m', 'u p', 'd

In [26]:
def process(mess):
    non_punc = [char for char in mess if char not in string.punctuation]
    non_punc = ''.join(non_punc)
    return " ".join([word for word in non_punc.split() if word.lower() not in stop])

In [33]:
review_user.rename(columns={'0':'business_id','1':'text','2':'rating','4':'user_id'}, inplace=True )

C:\Users\admin\AppData\Local\Temp\ipykernel_20728\1109201922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_user.rename(columns={'0':'business_id','1':'text','2':'rating','4':'user_id'}, inplace=True )


In [35]:
user_text = review_user[['user_id', 'text']]
business_text = review_user[['business_id', 'text']]

In [36]:
user_text = user_text.groupby('user_id').agg({'text': " ".join})
business_text = business_text.groupby('business_id').agg({'text': " ".join})

In [39]:
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(user_text['text'])
userid_vectors.shape

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(896, 4289)

In [40]:
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(business_text['text'])
businessid_vectors.shape

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(1318, 4289)

In [43]:
userid_rating_matrix = pd.pivot_table(review_user, values='rating', index=['user_id'], columns=['business_id'])
userid_rating_matrix.shape

(896, 1318)

In [44]:
P = pd.DataFrame(userid_vectors.toarray(), index=user_text.index, columns=userid_vectorizer.get_feature_names_out())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_text.index, columns=businessid_vectorizer.get_feature_names_out())

In [45]:
def matrix_factorization(R, P, Q, steps=25, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q

In [48]:
%%time
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

CPU times: total: 5min 54s
Wall time: 5min 55s


In [57]:
words = "i want to have dinner with beautiful views"
test_df= pd.DataFrame([words], columns=['text'])
test_df['text'] = test_df['text'].apply(process)
test_vectors = userid_vectorizer.transform(test_df['text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names_out())

predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:7]

for i in topRecommendations.index:
    print(df_business[df_business['0']==i]['2'].iloc[0])
    print(df_business[df_business['0']==i]['10'].iloc[0])
    print(str(df_business[df_business['0']==i]['9'].iloc[0])+ ' '+str(df_business[df_business['0']==i]['4'].iloc[0]))
    print('')

KJNWztNYuNysB1MVsX0FOA
0      False
3      False
5      False
6      False
7      False
       ...  
464    False
467    False
468    False
469    False
471    False
Name: 0, Length: 339, dtype: bool
qWkqQM3zcfS4zWPO32UXAA
0      False
3      False
5      False
6      False
7      False
       ...  
464    False
467    False
468    False
469    False
471    False
Name: 0, Length: 339, dtype: bool
oGTvgrrc-_EnuBQXyXTjng
0      False
3      False
5      False
6      False
7      False
       ...  
464    False
467    False
468    False
469    False
471    False
Name: 0, Length: 339, dtype: bool
blwpnVEzzao7VqnckTRV8w
0      False
3      False
5      False
6      False
7      False
       ...  
464    False
467    False
468    False
469    False
471    False
Name: 0, Length: 339, dtype: bool
XTX8l2ABSrEKfzZM9ViRRQ
0      False
3      False
5      False
6      False
7      False
       ...  
464    False
467    False
468    False
469    False
471    False
Name: 0, Length: 339, dtype: bool
